In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import time
from datetime import datetime
from scipy import integrate, optimize
import warnings
warnings.filterwarnings('ignore')
from scipy.integrate import odeint

In [3]:
cv  = pd.read_csv('drive/My Drive/Thesis/Data/cov_data_may.csv')
cv


,Date,Active Cases,New Deaths,New Recovered,New Cases,Total Deaths,Total Recovered,Total Cases
0,2021-05-01 00:00:00+00:00,100250,131,4344,4512,45652,1526978,1672880
1,2021-05-02 00:00:00+00:00,100760,144,3740,4394,45796,1530718,1677274
2,2021-05-03 00:00:00+00:00,100564,153,4773,4730,45949,1535491,1682004
3,2021-05-04 00:00:00+00:00,99087,188,5658,4369,46137,1541149,1686373
4,2021-05-05 00:00:00+00:00,98217,212,5943,5285,46349,1547092,1691658
5,2021-05-06 00:00:00+00:00,98277,147,5440,5647,46496,1552532,1697305
6,2021-05-07 00:00:00+00:00,98546,167,5891,6327,46663,1558423,1703632
7,2021-05-08 00:00:00+00:00,99003,179,5494,6130,46842,1563917,1709762
8,2021-05-09 00:00:00+00:00,98395,170,4360,3922,47012,1568277,1713684
9,2021-05-10 00:00:00+00:00,96742,206,6338,4891,47218,1574615,1718575


In [4]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def mse(predict,target):
    return ((predict-target)**2).mean()

def mae(predict,target):
    return (abs(target-predict)).mean()

def mape(predict,target):
    return (abs((target - predict) / target).mean()) * 100

SEIR Model

In [6]:
# SEIR MODEL - DIFFERENTIAL EQUATIONS
# The SEIR model adds an “Exposed” state for individuals that have contracted the disease but are not yet
# infectious. This uses the fact that there is an incubation stage of infection that lasts approx. 6 days.
# The model assumes people carry lifelong immunity to a disease upon recovery, which hasn't been disproven
# for COVID-19.

def comp(state, t, N, beta, gamma, sigma):
    S, E, I, R = state
    dSdt = -(beta * S * I)/N # Change in susceptible population over time
    dEdt = (beta * S * I)/N - sigma * E # Change in exposed population over time
    dIdt = sigma * E - gamma * I # Change in infectious population over time
    dRdt = gamma * I # Change in recovered population over time
    return dSdt, dEdt, dIdt, dRdt



Basic SEIR

In [8]:
total_pop = 276051328 
removed = 1572630 #recovered + dead
infected = 	100250		
exposed = 5000 #assumption
susceptible = total_pop - infected - removed - exposed 

last_day_total = 1668368 #total cases April 30, 2021

σ=1/5.2   
γ=1/2.3   
β=R0γ=2.2γ = 0.96  =>>>> 1535730.93 (MAE)

σ=1/5.2   
γ=1/2.3 
β=R = 1.4γ = 0.1 =>>>>  78241.47

σ=1/5.2   
γ=1/D = 1/14
β=R = 2.3γ  =>>>>> 42281.14

σ=1/5.2   
γ=1/D = 1/14
β=R = 1.4γ = 0.1 =>>>>  6639.98

In [20]:
real = cv['New Cases'][31:]
real

31    4824
32    5246
33    5353
34    6486
35    6594
36    5832
37    6993
Name: New Cases, dtype: int64

In [ ]:
# BETA => effective_contact_rate
# GAMMA => recovery_rate
# SIGMA => incubation_period (r0*gamma)
#Recovery rate, γ = 1/D, is determined by the average duration, D, of infection

days_to_recover = 14
recovery_rate = 1/2.3
incubation_period = 5.2 # incubation parameter

days = range(0,38)

# Computation
#calc = odeint(comp, [susceptible, exposed, infected, recovered], days,
#             args=(total_pop, 1/incubation_period, effective_contact_rate, recovery_rate))
#S, E, I, R = calc.T

R0 = 2.2
sigma= 1/incubation_period
gamma= recovery_rate
beta= R0*gamma

# Computation
calc = odeint(comp, [susceptible, exposed, infected, removed], days,
             args=(total_pop, beta, gamma, sigma))
S, E, I, R = calc.T
df = pd.DataFrame({'Susceptible': S, 'Exposed': E, 'Infected': I, 'Recovered': R, 'Day': days})


In [ ]:
t = df["Recovered"] + df["Infected"]
df['Total'] = t

df

,Susceptible,Exposed,Infected,Recovered,Day,Total
0,2.744191e+08,5.000000e+03,1.002500e+05,1.572630e+06,0,1.672880e+06
1,2.743392e+08,7.645250e+04,7.217232e+04,1.609182e+06,1,1.681354e+06
2,2.742761e+08,1.202850e+05,6.253697e+04,1.638013e+06,2,1.700550e+06
3,2.742175e+08,1.526206e+05,6.203455e+04,1.664863e+06,3,1.726897e+06
4,2.741567e+08,1.812123e+05,6.637123e+04,1.692647e+06,4,1.759018e+06
5,2.740905e+08,2.099197e+05,7.361255e+04,1.722993e+06,5,1.796605e+06
6,2.740163e+08,2.408093e+05,8.294020e+04,1.756958e+06,6,1.839898e+06
7,2.739324e+08,2.751430e+05,9.407093e+04,1.795375e+06,7,1.889446e+06
8,2.738371e+08,3.138447e+05,1.069859e+05,1.839016e+06,8,1.946002e+06
9,2.737288e+08,3.577252e+05,1.218055e+05,1.888681e+06,9,2.010487e+06


In [ ]:
ncArr = []
for data in df['Total']:
    calc = data - last_day_total
    ncArr.append(calc)
    last_day_total = data

ncArr_train =ncArr[:31]

ncArr_test =ncArr[-7:]
ncArr_test

[1060691.349221306,
 1194704.2196931094,
 1343512.959746454,
 1508167.3720128834,
 1689628.405257471,
 1888712.9587332718,
 2106027.282844633]

In [ ]:
print('MAE : ', mae( real, ncArr_test))
print('MAPE : ', mape (real, ncArr_test))

MAE :  1535730.9353584468
MAPE :  99.60664490504669


#################################################

In [ ]:
# BETA => effective_contact_rate
# GAMMA => recovery_rate
# SIGMA => incubation_period (r0*gamma)

days_to_recover = 14
recovery_rate = 1/days_to_recover
incubation_period = 5.2 # incubation parameter

days = range(0,38)

R0 = 2.2
sigma= 1/incubation_period
gamma= recovery_rate
beta= R0*gamma

# Computation
calc = odeint(comp, [susceptible, exposed, infected, removed], days,
             args=(total_pop, beta, gamma, sigma))
S, E, I, R = calc.T
df = pd.DataFrame({'Susceptible': S, 'Exposed': E, 'Infected': I, 'Recovered': R, 'Day': days})

In [ ]:
t = df["Recovered"] + df["Infected"]
df['Total'] = t

df

,Susceptible,Exposed,Infected,Recovered,Day,Total
0,2.744191e+08,5000.000000,100250.000000,1.572630e+06,0,1.672880e+06
1,2.744038e+08,17992.457270,95537.032975,1.579606e+06,1,1.675143e+06
2,2.743891e+08,28230.423302,93287.591256,1.586337e+06,2,1.679624e+06
3,2.743746e+08,36499.362544,92898.158473,1.592976e+06,3,1.685875e+06
4,2.743600e+08,43374.344068,93931.821616,1.599642e+06,4,1.693573e+06
5,2.743452e+08,49277.080301,96073.370636,1.606422e+06,5,1.702495e+06
6,2.743300e+08,54517.537309,99096.555130,1.613387e+06,6,1.712484e+06
7,2.743142e+08,59324.290689,102840.210481,1.620595e+06,7,1.723436e+06
8,2.742978e+08,63866.676438,107190.851375,1.628093e+06,8,1.735284e+06
9,2.742807e+08,68270.951642,112069.987856,1.635921e+06,9,1.747991e+06


In [ ]:
ncArr = []
for data in df['Total']:
    calc = data - last_day_total
    ncArr.append(calc)
    last_day_total = data

ncArr_train =ncArr[:31]

ncArr_test =ncArr[-7:]
ncArr_test

[41091.438705872744,
 43257.36105389474,
 45536.468193122186,
 47934.58848167956,
 50457.83541962458,
 53112.6217626296,
 55905.67343191244]

In [ ]:
print('MAE : ', mae( ncArr_test, real))
print('MAPE : ', mape (ncArr_test, real))

MAE :  42281.141006962265
MAPE :  720.211424133393


#######################################################

In [ ]:
# BETA => effective_contact_rate
# GAMMA => recovery_rate
# SIGMA => incubation_period (r0*gamma)

days_to_recover = 14
recovery_rate = 1/days_to_recover
incubation_period = 5.2 # incubation parameter

days = range(0,38)

R0 = 1.4
sigma= 1/incubation_period
gamma= recovery_rate
beta= R0*gamma

# Computation
calc = odeint(comp, [susceptible, exposed, infected, removed], days,
             args=(total_pop, beta, gamma, sigma))
S, E, I, R = calc.T
df = pd.DataFrame({'Susceptible': S, 'Exposed': E, 'Infected': I, 'Recovered': R, 'Day': days})

In [ ]:
t = df["Recovered"] + df["Infected"]
df['Total'] = t

df

,Susceptible,Exposed,Infected,Recovered,Day,Total
0,2.744191e+08,5000.000000,100250.000000,1.572630e+06,0,1.672880e+06
1,2.744094e+08,12933.949444,95043.633272,1.579593e+06,1,1.674637e+06
2,2.744001e+08,19089.526359,91493.900377,1.586247e+06,2,1.677741e+06
3,2.743912e+08,23908.133665,89199.907456,1.592694e+06,3,1.681894e+06
4,2.743824e+08,27722.508375,87859.642544,1.599012e+06,4,1.686872e+06
5,2.743737e+08,30783.490553,87245.621782,1.605262e+06,5,1.692508e+06
6,2.743650e+08,33280.198781,87186.533796,1.611489e+06,6,1.698676e+06
7,2.743563e+08,35355.234908,87553.406177,1.617728e+06,7,1.705281e+06
8,2.743476e+08,37116.143638,88249.179239,1.624005e+06,8,1.712254e+06
9,2.743388e+08,38644.049188,89200.848106,1.630341e+06,9,1.719542e+06


In [ ]:
ncArr = []
for data in df['Total']:
    calc = data - last_day_total
    ncArr.append(calc)
    last_day_total = data

ncArr_train =ncArr[:31]

ncArr_test =ncArr[-7:]
ncArr_test

[11842.504948667949,
 12069.136625356274,
 12299.990307378117,
 12535.160730971256,
 12774.740336823277,
 13018.820021527587,
 13267.490048825508]

In [ ]:
print('MAE : ', mae( real, ncArr_test))
print('MAPE : ', mape (real, ncArr_test))

MAE :  6639.977574221424
MAPE :  53.059194362035456


#######################################################

In [ ]:
# BETA => effective_contact_rate
# GAMMA => recovery_rate
# SIGMA => incubation_period (r0*gamma)

days_to_recover = 14
recovery_rate = 1/2.3
incubation_period = 5.2 # incubation parameter

days = range(0,38)

R0 = 1.4
sigma= 1/incubation_period
gamma= recovery_rate
beta= R0*gamma

# Computation
calc = odeint(comp, [susceptible, exposed, infected, removed], days,
             args=(total_pop, beta, gamma, sigma))
S, E, I, R = calc.T
df = pd.DataFrame({'Susceptible': S, 'Exposed': E, 'Infected': I, 'Recovered': R, 'Day': days})

In [ ]:
t = df["Recovered"] + df["Infected"]
df['Total'] = t

df

,Susceptible,Exposed,Infected,Recovered,Day,Total
0,2.744191e+08,5000.000000,100250.000000,1.572630e+06,0,1.672880e+06
1,2.743688e+08,49653.694170,69761.680012,1.608804e+06,1,1.678565e+06
2,2.743315e+08,74710.794369,55177.457667,1.635563e+06,2,1.690740e+06
3,2.743004e+08,89912.454832,48732.045992,1.657946e+06,3,1.706678e+06
4,2.742717e+08,100220.157651,46476.365146,1.678537e+06,4,1.725013e+06
5,2.742437e+08,108164.185747,46403.228094,1.698672e+06,5,1.745075e+06
6,2.742154e+08,115038.159867,47493.687100,1.719054e+06,6,1.766548e+06
7,2.741862e+08,121506.757222,49231.788331,1.740064e+06,7,1.789295e+06
8,2.741558e+08,127914.927139,51357.658383,1.761920e+06,8,1.813277e+06
9,2.741240e+08,134445.239215,53741.864514,1.784760e+06,9,1.838501e+06


In [ ]:
ncArr = []
for data in df['Total']:
    calc = data - last_day_total
    ncArr.append(calc)
    last_day_total = data

ncArr_train =ncArr[:31]

ncArr_test =ncArr[-7:]
ncArr_test

[72658.53129674308,
 76195.4650114975,
 79899.97911323467,
 83779.54276292678,
 87841.92650059704,
 92095.1891095615,
 96547.66843466228]

In [ ]:
print('MAE : ', mae( ncArr_test, real))
print('MAPE : ', mape (ncArr_test, real))

MAE :  78241.47174703183
MAPE :  1333.553427123341


##########################################################################

In [ ]:
# BETA => effective_contact_rate
# GAMMA => recovery_rate
# SIGMA => incubation_period (r0*gamma)

days_to_recover = 14
recovery_rate = 0.1
incubation_period = 5.2 # incubation parameter

days = range(0,38)

R0 = 2.2
sigma= 1
gamma= recovery_rate
beta= R0*gamma

# Computation
calc = odeint(comp, [susceptible, exposed, infected, removed], days,
             args=(total_pop, beta, gamma, sigma))
S, E, I, R = calc.T
df = pd.DataFrame({'Susceptible': S, 'Exposed': E, 'Infected': I, 'Recovered': R, 'Day': days})

In [ ]:
t = df["Recovered"] + df["Infected"]
df['Total'] = t

df

,Susceptible,Exposed,Infected,Recovered,Day,Total
0,2.744191e+08,5000.000000,1.002500e+05,1.572630e+06,0,1.672880e+06
1,2.743972e+08,15678.310059,1.014387e+05,1.582628e+06,1,1.684067e+06
2,2.743743e+08,20371.145073,1.092068e+05,1.593129e+06,2,1.702335e+06
3,2.743493e+08,23412.246475,1.197282e+05,1.604559e+06,3,1.724287e+06
4,2.743218e+08,26123.967020,1.319291e+05,1.617129e+06,4,1.749058e+06
5,2.742915e+08,28922.371375,1.455738e+05,1.630993e+06,5,1.776566e+06
6,2.742580e+08,31952.057769,1.606880e+05,1.646293e+06,6,1.806981e+06
7,2.742211e+08,35277.725916,1.773864e+05,1.663183e+06,7,1.840569e+06
8,2.741804e+08,38942.125776,1.958213e+05,1.681828e+06,8,1.877649e+06
9,2.741354e+08,42983.739113,2.161684e+05,1.702411e+06,9,1.918579e+06


In [ ]:
ncArr = []
for data in df['Total']:
    calc = data - last_day_total
    ncArr.append(calc)
    last_day_total = data

ncArr_train =ncArr[:31]

ncArr_test =ncArr[-7:]
ncArr_test

[351392.1622095676,
 386627.66359717306,
 425241.83874915633,
 467525.75474606175,
 513788.46478122286,
 564356.544792356,
 619573.1482920637]

In [ ]:
print('MAE : ', mae( ncArr_test, real))
print('MAPE : ', mape (ncArr_test, real))

MAE :  469596.79673822876
MAPE :  7919.282612906202


################################################################

In [ ]:
# BETA => effective_contact_rate
# GAMMA => recovery_rate
# SIGMA => incubation_period (r0*gamma)

days_to_recover = 14
recovery_rate = 0.1
incubation_period = 5.2 # incubation parameter

days = range(0,38)

R0 = 1.4
sigma= 1
gamma= recovery_rate
beta= R0*gamma

# Computation
calc = odeint(comp, [susceptible, exposed, infected, removed], days,
             args=(total_pop, beta, gamma, sigma))
S, E, I, R = calc.T
df = pd.DataFrame({'Susceptible': S, 'Exposed': E, 'Infected': I, 'Recovered': R, 'Day': days})

In [ ]:
t = df["Recovered"] + df["Infected"]
df['Total'] = t

df

,Susceptible,Exposed,Infected,Recovered,Day,Total
0,2.744191e+08,5000.000000,100250.000000,1.572630e+06,0,1.672880e+06
1,2.744053e+08,10534.066221,98591.900054,1.582526e+06,1,1.681117e+06
2,2.743915e+08,12626.787528,100389.280747,1.592459e+06,2,1.692848e+06
3,2.743773e+08,13623.782748,103379.288168,1.602642e+06,3,1.706021e+06
4,2.743627e+08,14279.709536,106837.066003,1.613150e+06,4,1.719987e+06
5,2.743476e+08,14837.632517,110532.429911,1.624017e+06,5,1.734549e+06
6,2.743319e+08,15376.115740,114394.366836,1.635262e+06,6,1.749656e+06
7,2.743158e+08,15920.837458,118403.150746,1.646901e+06,7,1.765304e+06
8,2.742990e+08,16480.483018,122555.652848,1.658947e+06,8,1.781503e+06
9,2.742817e+08,17058.287331,126854.189687,1.671417e+06,9,1.798271e+06


In [ ]:
last_day_total = 1668368 #total cases April 30, 2021
ncArr = []
for data in df['Total']:
    calc = data - last_day_total
    ncArr.append(calc)
    last_day_total = data

ncArr_train =ncArr[:31]

ncArr_test =ncArr[-7:]
ncArr_test

[35655.31176814623,
 36892.56252957042,
 38171.94975170866,
 39494.942821721546,
 40862.846612642985,
 42277.19824953284,
 43739.48413503403]

In [ ]:
print('MAE : ', mae( ncArr_test, real))
print('MAPE : ', mape (ncArr_test, real))

MAE :  33680.89940976525
MAPE :  576.3550771728643


SEIR + LSTM ###################################################################

In [35]:
# BETA => effective_contact_rate
# GAMMA => recovery_rate
# SIGMA => incubation_period (r0*gamma)

days_to_recover = 14
recovery_rate = 1/days_to_recover
incubation_period = 5.2 # incubation parameter

days = range(0,31)

R0 = 1.4
sigma= 1/incubation_period
gamma= recovery_rate
beta= R0*gamma

# Computation
calc = odeint(comp, [susceptible, exposed, infected, removed], days,
             args=(total_pop, beta, gamma, sigma))
S, E, I, R = calc.T
df = pd.DataFrame({'Susceptible': S, 'Exposed': E, 'Infected': I, 'Recovered': R, 'Day': days})


In [36]:
t = df["Recovered"] + df["Infected"]
df['Total'] = t

df

,Susceptible,Exposed,Infected,Recovered,Day,Total
0,2.743734e+08,5000.000000,100250.000000,1.572630e+06,0,1.672880e+06
1,2.743638e+08,12932.479392,95043.489756,1.579593e+06,1,1.674637e+06
2,2.743545e+08,19086.883960,91493.380018,1.586247e+06,2,1.677740e+06
3,2.743455e+08,23904.524842,89198.839174,1.592694e+06,3,1.681893e+06
4,2.743367e+08,27718.073270,87857.898758,1.599012e+06,4,1.686870e+06
5,2.743280e+08,30778.322248,87243.105054,1.605262e+06,5,1.692505e+06
6,2.743194e+08,33274.356762,87183.167349,1.611489e+06,6,1.698672e+06
7,2.743107e+08,35348.754712,87549.127194,1.617727e+06,7,1.705276e+06
8,2.743020e+08,37109.043755,88243.934140,1.624004e+06,8,1.712248e+06
9,2.742932e+08,38636.335985,89194.589233,1.630339e+06,9,1.719534e+06


In [37]:
last_day_total = 1668368 #total cases April 30, 2021

In [38]:
ncArr = []
for data in df['Total']:
    calc = data - last_day_total
    ncArr.append(calc)
    last_day_total = data

ncArr_train =ncArr
ncArr_train

[4512.0,
 1756.9781865738332,
 3103.3782338702586,
 4152.270851462847,
 4977.585435188143,
 5635.053960043471,
 6166.692980903434,
 6604.182700217003,
 6971.413012586767,
 7286.402667571558,
 7562.745964951348,
 7810.702330403728,
 8038.018101827474,
 8250.545968624065,
 8452.711441037478,
 8647.864728246117,
 8838.545818153303,
 9026.684315129183,
 9213.750067934161,
 9400.866680340609,
 9588.896999804769,
 9778.507806899725,
 9970.218271656195,
 10164.43656584411,
 10361.487596732797,
 10561.633802249562,
 10765.090879288968,
 10972.039498776197,
 11182.634407968959,
 11397.01102646091,
 11615.290623645997]

In [39]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [40]:
from pandas import DataFrame
new_cases = DataFrame (ncArr_train)
covid_data = new_cases.values.astype(float)
train_data = covid_data

In [41]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1,1))
train_data_normalized = scaler.fit_transform(train_data.reshape(-1,1))
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)
train_window = 7

def create_inout_sequences(input_data, tw):
  inout_seq = []
  L = len(input_data)
  for i in range(L-tw):
    train_seq = input_data[i:i+tw]
    train_label = input_data[i+tw:i+tw+1]
    inout_seq.append((train_seq, train_label))
  return inout_seq

train_inout_seq = create_inout_sequences(train_data_normalized, train_window)

In [42]:
#Model
class LSTM(nn.Module):
  def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
    super().__init__()
    self.hidden_layer_size = hidden_layer_size
    self.lstm = nn.LSTM(input_size, hidden_layer_size)
    self.linear = nn.Linear(hidden_layer_size, output_size)
    self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                        torch.zeros(1,1,self.hidden_layer_size))
    
  def forward(self, input_seq):
    lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
    predictions = self.linear(lstm_out.view(len(input_seq), -1))
    return predictions[-1]

model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [65]:
#Training
epochs = 1000

for i in range(epochs):
  for seq, labels in train_inout_seq:
    optimizer.zero_grad()
    model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                         torch.zeros(1, 1, model.hidden_layer_size))
    
    y_pred = model(seq)

    single_loss = loss_function(y_pred, labels)
    single_loss.backward()
    optimizer.step()

  if i%100==1:
    print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')


epoch:   1 loss: 0.00000231
epoch: 101 loss: 0.00000133
epoch: 201 loss: 0.00000000
epoch: 301 loss: 0.00000038
epoch: 401 loss: 0.00000000
epoch: 501 loss: 0.00000447
epoch: 601 loss: 0.00000936
epoch: 701 loss: 0.00000000
epoch: 801 loss: 0.00000006
epoch: 901 loss: 0.00002773
epoch: 999 loss: 0.0000007896


In [66]:
#Prediction
fut_pred = 7

test_inputs = train_data_normalized[-train_window:].tolist()
print(test_inputs)

model.eval()

for i in range(fut_pred):
  seq = torch.FloatTensor(test_inputs[-train_window:])
  with torch.no_grad():
    model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                    torch.zeros(1, 1, model.hidden_layer_size))
    test_inputs.append(model(seq).item())

test_inputs[fut_pred:]
actual_predictions = scaler.inverse_transform(np.array(test_inputs[train_window:] ).reshape(-1, 1))
print(actual_predictions)

[0.7456353902816772, 0.7862399220466614, 0.8275161981582642, 0.8695007562637329, 0.912225067615509, 0.9557166695594788, 1.0]
[[11831.2575727 ]
 [12051.74611307]
 [12279.98805142]
 [12516.95586775]
 [12763.90644105]
 [13022.78590654]
 [13296.4547069 ]]


In [67]:
test_data = real.to_list()
test_data

[4824, 5246, 5353, 6486, 6594, 5832, 6993]

Prediction SEIR + LSTM

In [ ]:
print('MAE : ', mae( test_data, actual_predictions))
print('MAPE : ', mape (test_data, actual_predictions))

MAE :  6585.006893260124
MAPE :  52.67710437719917


In [55]:
print('MAE : ', mae( test_data, actual_predictions))
print('MAPE : ', mape (test_data, actual_predictions))

MAE :  6627.841401338096
MAPE :  52.821395601356905


In [58]:
print('MAE : ', mae( test_data, actual_predictions))
print('MAPE : ', mape (test_data, actual_predictions))

MAE :  6630.941333497696
MAPE :  52.83641081703748


In [64]:
print('MAE : ', mae( test_data, actual_predictions))
print('MAPE : ', mape (test_data, actual_predictions))

MAE :  6610.821028606299
MAPE :  52.76119458874731


In [61]:
print('MAE : ', mae( test_data, actual_predictions))
print('MAPE : ', mape (test_data, actual_predictions))

MAE :  6611.945193665773
MAPE :  52.76354838733436


############################################

SEIR + BERT + LSTM

In [69]:
#Downloading libraries
!pip install -U sentence-transformers

     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 2.6 MB 18.8 MB/s 
     |████████████████████████████████| 1.2 MB 58.6 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 895 kB 64.8 MB/s 
     |████████████████████████████████| 3.3 MB 62.3 MB/s 
     |████████████████████████████████| 636 kB 54.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126709 sha256=97aa4f376901a367c13192876ea28ee3d518be413691f3a1e4b309cce7a1ff82
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [70]:
pip install fasttext

     |████████████████████████████████| 68 kB 4.8 MB/s  eta 0:00:01
  Using cached pybind11-2.7.0-py2.py3-none-any.whl (199 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3093380 sha256=411247e1e62ed3acd2d4e12d6108d1097874cea4760c1c360ca0b581696da39e
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [71]:
#Importing dependencies
import gensim
from gensim.models import KeyedVectors
from gensim import corpora
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from sklearn.metrics import silhouette_score
from wordcloud import WordCloud
from gensim.models.coherencemodel import CoherenceModel
import os
from sentence_transformers import SentenceTransformer
import copy
from datetime import datetime
import warnings
from matplotlib import pyplot as plt
from wordcloud import WordCloud
import contextlib
import spacy
import re
import random
import math
from gensim.utils import simple_preprocess
import fasttext.util


#word2vec, glove, fasttext
class WordVecVectorizer(object):
    def __init__(self, model, method):
        self.model = model
        self.method = method

        if method == 'WORD2VEC':
          self.dim = 100 
                     
        elif method == 'GLOVE':
           self.dim = 5 
           
        elif method == 'FASTTEXT':           
            self.dim = 300
        else:
          self.dim = 200
       
        
    def fit(self, X, y):
        return self    
        
    def transform(self, X):
        return np.array([
            np.mean([self.model[w] for w in texts.split() if w in self.model]
                    or [np.zeros(self.dim)], axis=0)
            for texts in X              
        ])

In [72]:
def loadGloveModel(File):
    print("Loading Glove Model")
    f = open(File,'r')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel



def vectorize(method, sentences):
  if method == 'BOW':
    print('Getting vector representations for BoW ...')
    dictionary = corpora.Dictionary()
    doc_tokenized = [simple_preprocess(doc) for doc in sentences]
    BoW_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in doc_tokenized]
    vec = np.array(BoW_corpus)
    print('Getting vector representations for BoW. Done!')
    return vec

  elif method == 'BERT':
    print('Getting vector representations for BERT ...')
    model = SentenceTransformer('indobenchmark/indobert-base-p1')
    vec = np.array(model.encode(sentences, show_progress_bar=True))
    print('Getting vector representations for BERT. Done!')
    return vec

  elif method == 'WORD2VEC':
     print('Getting vector representations for WORD2VEC ...')
     model = gensim.models.KeyedVectors.load_word2vec_format('drive/My Drive/Thesis/pretrained_model/word2vec/model.bin', binary=True) 
     wtv_vect = WordVecVectorizer(model, method)
     embeddings = wtv_vect.transform(sentences)
     vec = np.vstack(embeddings)
     print('Getting vector representations for WORD2VEC. Done!')
     return vec

  elif method == 'GLOVE':
    print('Getting vector representations for GLOVE ...')
    model = loadGloveModel("drive/My Drive/Thesis/Data/gensim_glove_vectors.txt") 
    wtv_vect = WordVecVectorizer(model, method)
    embeddings = wtv_vect.transform(sentences)
    vec = np.vstack(embeddings)
    print('Getting vector representations for GLOVE. Done!')
    return vec

  elif method == 'FASTTEXT':
    print('Getting vector representations for FASTTEXT ...')
    #loading the model
    model = fasttext.load_model('drive/My Drive/Thesis/pretrained_model/fasttext/model.bin') 
    wtv_vect = WordVecVectorizer(model, method)
    embeddings = wtv_vect.transform(sentences)
    vec = np.vstack(embeddings)
    print('Getting vector representations for FASTTEXT. Done!')
    return vec       

In [80]:
#loading covid data 
new_cases = cv['New Cases']


In [102]:
train_data = ncArr_train
cv_feature = train_data

In [101]:
#loading tweet data
tw  = pd.read_csv('drive/My Drive/Thesis/Data/Tweet_TM/task1_may_june/bert_cluster_0_combined.csv')
tw_feature = tw['clean_sentences']


In [103]:
#create vector for tweets
method = 'BERT'
tweet_feature  = vectorize(method, tw_feature)


Getting vector representations for BERT ...



Getting vector representations for BERT. Done!


In [104]:
#concatenating features

features = nn.Sequential(
            nn.Linear(768, 1),
            nn.ReLU(),
        )

x1 = torch.Tensor(cv_feature)
x2 = features(torch.Tensor(tweet_feature))
x1 = x1.view(31, 1) 
x = torch.cat((x1, x2),1)

x = x.detach().numpy()
train_data = np.array([])
for x in x:
  train_data = np.append(train_data, x[0])
train_data.shape

(31,)

In [235]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1,1))
train_data_normalized = scaler.fit_transform(train_data.reshape(-1,1))
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)
train_window = 7

train_inout_seq = create_inout_sequences(train_data_normalized, train_window)

#MODEL
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

#Training
epochs = 1000

for i in range(epochs):
  for seq, labels in train_inout_seq:
    optimizer.zero_grad()
    model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                         torch.zeros(1, 1, model.hidden_layer_size))
    
    y_pred = model(seq)

    single_loss = loss_function(y_pred, labels)
    single_loss.backward()
    optimizer.step()

  if i%100==1:
    print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 0.21479741
epoch: 101 loss: 0.00021077
epoch: 201 loss: 0.00002840
epoch: 301 loss: 0.00012081
epoch: 401 loss: 0.00001702
epoch: 501 loss: 0.00000202
epoch: 601 loss: 0.00000344
epoch: 701 loss: 0.00003507
epoch: 801 loss: 0.00000399
epoch: 901 loss: 0.00000026
epoch: 999 loss: 0.0000004294


In [236]:
#Prediction
test_data = new_cases[31:]
fut_pred = 7

test_inputs = train_data_normalized[-train_window:].tolist()
print(test_inputs)

model.eval()

for i in range(fut_pred):
  seq = torch.FloatTensor(test_inputs[-train_window:])
  with torch.no_grad():
    model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                    torch.zeros(1, 1, model.hidden_layer_size))
    test_inputs.append(model(seq).item())

test_inputs[fut_pred:]
actual_predictions = scaler.inverse_transform(np.array(test_inputs[train_window:] ).reshape(-1, 1))
print(actual_predictions)


[0.7456352114677429, 0.7862398624420166, 0.8275160789489746, 0.8695005774497986, 0.912225067615509, 0.9557164907455444, 1.0]
[[11821.75528688]
 [12023.04514165]
 [12221.18251579]
 [12413.03843271]
 [12596.23016939]
 [12767.93900271]
 [12924.44659205]]


In [125]:
cov = new_cases[31:]

print('MAE : ', mae( cov, actual_predictions.reshape(7,)))
print('MAPE : ', mape (cov, actual_predictions.reshape(7,)))

MAE :  6563.810842533419
MAPE :  52.768112224196926


In [135]:
cov = new_cases[31:]

print('MAE : ', mae( cov, actual_predictions.reshape(7,)))
print('MAPE : ', mape (cov, actual_predictions.reshape(7,)))

MAE :  6626.995771377661
MAPE :  53.00155840375462


In [210]:
cov = new_cases[31:]

print('MAE : ', mae( cov, actual_predictions.reshape(7,)))
print('MAPE : ', mape (cov, actual_predictions.reshape(7,)))

MAE :  6651.424873068948
MAPE :  53.10233711719034


In [225]:
cov = new_cases[31:]

print('MAE : ', mae( cov, actual_predictions.reshape(7,)))
print('MAPE : ', mape (cov, actual_predictions.reshape(7,)))

MAE :  6399.33303002671
MAPE :  52.06877575935707


In [237]:
cov = new_cases[31:]

print('MAE : ', mae( cov, actual_predictions.reshape(7,)))
print('MAPE : ', mape (cov, actual_predictions.reshape(7,)))

MAE :  6491.376734454922
MAPE :  52.48226063192003


6546.59  52.68

###################################

##############################################################################